In [1]:
import random
import quimb as qu
import quimb.tensor as qtn
import numpy as np

In [2]:
N = 2
circ = qtn.Circuit(N)

In [3]:
def brickwork_state_init(qtm_circ):
    # hamadard on all of the qubits
    for i in range(2):
        qtm_circ.apply_gate('H', i)
        print(np.round(circ.psi.to_qarray(), 10))
    # create the entanglement for the brickwork state
    qtm_circ.apply_gate('CZ', 0, 1)
    return qtm_circ
# initiate brickwork state

In [4]:
#circ.apply_gate('X',0) 
print(np.nonzero(np.round(circ.psi.to_qarray(), 10)))

(array([0]), array([0]))


In [5]:
print(np.round(circ.psi.to_qarray(), 10))

[[1.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]]


In [6]:
circ = brickwork_state_init(circ)

[[0.707107+0.j]
 [0.      +0.j]
 [0.707107+0.j]
 [0.      +0.j]]
[[0.5+0.j]
 [0.5+0.j]
 [0.5+0.j]
 [0.5+0.j]]


In [7]:
print(np.round(circ.psi.to_qarray(), 10))

[[ 0.5+0.j]
 [ 0.5+0.j]
 [ 0.5+0.j]
 [-0.5+0.j]]


In [8]:
fh = np.matmul(np.array([[1, 1], [1,-1]]), np.array([[1, 0], [0,0]]))
sh = np.matmul(fh, np.array([[1, 1], [1,-1]]))
proj = 1/2 * sh
def rotation_z_gate(theta):
    rot_z = np.array([[np.exp(-1j*theta), 0],[0, np.exp(1j*theta)]])
    return rot_z

def z_rot_proj_meas(qubit, theta, qtm_circ):
    # change basis
    #irc.apply_gate('H', qubit)
    z_rot = rotation_z_gate(theta)

    #qtm_circ.apply_gate(z_rot, qubit)
    qtm_circ.apply_gate(proj, qubit)
    print(qubit)
    print(z_rot)
    # renormalize
    state_vector = np.round(qtm_circ.psi.to_qarray(), 10)
    renorm_factor = np.sqrt(np.dot(state_vector.conj().T, state_vector))
    print(renorm_factor)
    renorm_matrix = np.array([[1, 0], [0, 1]])* 1/renorm_factor
    qtm_circ.apply_gate(renorm_matrix, qubit)
    
    

In [9]:
for i in range(1):
    z_rot_proj_meas(i, 0, circ)

0
[[1.+0.j 0.+0.j]
 [0.+0.j 1.+0.j]]
[[0.707107+0.j]]


In [10]:
 print(np.nonzero(np.round(circ.psi.to_qarray(), 10)))

(array([0, 2]), array([0, 0]))


In [11]:
print(np.round(circ.psi.to_qarray(), 10))

[[0.707107+0.j]
 [0.      +0.j]
 [0.707107+0.j]
 [0.      +0.j]]


In [12]:
circ.apply_gate('H', 0)
print(np.round(circ.psi.to_qarray(), 10))

[[ 1.+0.j]
 [ 0.+0.j]
 [-0.+0.j]
 [-0.+0.j]]
